In [269]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup as BS
from bs4 import Comment
import time
from tqdm import tqdm
import numpy as np

In [198]:
odds_2011 = pd.read_excel('data/mlb odds 2012.xlsx')

In [199]:
odds_2011['away_team'] = odds_2011.Team.shift(1)

In [200]:
odds_2011['away_open'] = odds_2011['Open'].shift(1)

In [201]:
odds_2011 = odds_2011.drop(columns = ['1st','2nd','3rd','4th','5th','6th','7th','8th','9th','Open OU','Unnamed: 18','Unnamed: 20', 'Close','Close OU' ])

In [202]:
odds_2011

,Date,Rot,VH,Team,Pitcher,Final,Open,away_team,away_open
0,328,947,V,SEA,FHERNANDZ-R,3,-123,NaN,NaN
1,328,948,H,OAK,BMCCARTHY-R,1,103,SEA,-123.0
2,329,995,V,SEA,JVARGAS-L,1,-110,OAK,103.0
3,329,996,H,OAK,BCOLON-R,4,-110,SEA,-110.0
4,404,997,V,STL,KLOHSE-R,4,153,OAK,-110.0
...,...,...,...,...,...,...,...,...,...
4929,1025,904,H,SFO,BUMGARNER-L,2,100,DET,-120.0
4930,1027,905,V,SFO,VOGELSONG-R,2,120,SFO,100.0
4931,1027,906,H,DET,ASANCHEZ-R,0,-140,SFO,120.0
4932,1028,907,V,SFO,MCAIN-R,4,115,DET,-140.0


In [203]:
odds_2011['away_score'] = odds_2011['Final'].shift(1)

In [204]:
odds_2011[odds_2011['VH']=='H']

,Date,Rot,VH,Team,Pitcher,Final,Open,away_team,away_open,away_score
1,328,948,H,OAK,BMCCARTHY-R,1,103,SEA,-123.0,3.0
3,329,996,H,OAK,BCOLON-R,4,-110,SEA,-110.0,1.0
5,404,998,H,MIA,JJOHNSON-R,1,-173,STL,153.0,4.0
7,405,902,H,NYM,JSANTANA-L,1,-105,ATL,-105.0,0.0
9,405,904,H,PIT,EBEDARD-L,0,176,PHI,-196.0,1.0
...,...,...,...,...,...,...,...,...,...,...
4925,1022,928,H,SFO,MCAIN-R,9,-147,STL,127.0,0.0
4927,1024,902,H,SFO,BZITO-L,8,130,DET,-150.0,3.0
4929,1025,904,H,SFO,BUMGARNER-L,2,100,DET,-120.0,0.0
4931,1027,906,H,DET,ASANCHEZ-R,0,-140,SFO,120.0,2.0


In [205]:
def get_odds(filepath):
    df = pd.read_excel(filepath)
    df['away_team'] = df.Team.shift(1)
    df['away_open'] = df['Open'].shift(1)
    df = df.drop(columns = ['1st','2nd','3rd','4th','5th','6th','7th',
                                          '8th','9th','Open OU','Unnamed: 18','Unnamed: 20', 'Close','Close OU' ])
    df['away_score'] = df['Final'].shift(1)
    df = df[df['VH'] == 'H']
    
    return df

In [206]:
file_paths = [#'data/mlb odds 2012.xlsx',
             'data/mlb odds 2013.xlsx',
             'data/mlb odds 2014.xlsx',
             'data/mlb odds 2015.xlsx',
             'data/mlb odds 2016.xlsx',
             'data/mlb odds 2017.xlsx',
             'data/mlb odds 2018.xlsx',
             'data/mlb odds 2019.xlsx',
             'data/mlb odds 2020.xlsx',
             'data/mlb odds 2021.xlsx']

In [207]:
file_paths[1].split(' ')[2][0:4]

'2014'

In [208]:
odds = get_odds('data/mlb odds 2012.xlsx')
odds['year'] = '2012'
temp = []

for filepath in file_paths:
    try:
        df = get_odds(filepath)
        df['year'] = filepath.split(' ')[2][0:4]
        odds = odds.append(df)
    except KeyError:
        print(filepath)

data/mlb odds 2020.xlsx
data/mlb odds 2021.xlsx


In [209]:
odds

,Date,Rot,VH,Team,Pitcher,Final,Open,away_team,away_open,away_score,year,Run Line,Unnamed: 22
1,328,948,H,OAK,BMCCARTHY-R,1,103,SEA,-123.0,3.0,2012,NaN,NaN
3,329,996,H,OAK,BCOLON-R,4,-110,SEA,-110.0,1.0,2012,NaN,NaN
5,404,998,H,MIA,JJOHNSON-R,1,-173,STL,153.0,4.0,2012,NaN,NaN
7,405,902,H,NYM,JSANTANA-L,1,-105,ATL,-105.0,0.0,2012,NaN,NaN
9,405,904,H,PIT,EBEDARD-L,0,176,PHI,-196.0,1.0,2012,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4919,1025,906,H,WAS,ASANCHEZ-R,1,110,HOU,-121,4.0,2019,1.5,-110.0
4921,1026,908,H,WAS,PCORBIN-L,1,-115,HOU,105,8.0,2019,-1.5,100.0
4923,1027,910,H,WAS,MSCHERZER-R,1,140,HOU,-155,7.0,2019,1.5,-115.0
4925,1029,912,H,HOU,JVERLANDER-R,2,-180,WAS,162,7.0,2019,-1.5,-130.0


In [210]:
odds.drop(columns = ['Run Line', 'Unnamed: 22'])

,Date,Rot,VH,Team,Pitcher,Final,Open,away_team,away_open,away_score,year
1,328,948,H,OAK,BMCCARTHY-R,1,103,SEA,-123.0,3.0,2012
3,329,996,H,OAK,BCOLON-R,4,-110,SEA,-110.0,1.0,2012
5,404,998,H,MIA,JJOHNSON-R,1,-173,STL,153.0,4.0,2012
7,405,902,H,NYM,JSANTANA-L,1,-105,ATL,-105.0,0.0,2012
9,405,904,H,PIT,EBEDARD-L,0,176,PHI,-196.0,1.0,2012
...,...,...,...,...,...,...,...,...,...,...,...
4919,1025,906,H,WAS,ASANCHEZ-R,1,110,HOU,-121,4.0,2019
4921,1026,908,H,WAS,PCORBIN-L,1,-115,HOU,105,8.0,2019
4923,1027,910,H,WAS,MSCHERZER-R,1,140,HOU,-155,7.0,2019
4925,1029,912,H,HOU,JVERLANDER-R,2,-180,WAS,162,7.0,2019


In [211]:
odds_2020 = pd.read_excel('data/mlb odds 2020.xlsx')

In [212]:
odds_2020 = pd.read_excel(filepath)
odds_2020['away_team'] = odds_2020.Team.shift(1)
odds_2020['away_open'] = odds_2020['Open'].shift(1)
odds_2020['away_score'] = odds_2020['Final'].shift(1)
odds_2020 = odds_2020[odds_2020['VH'] == 'H']
odds_2020 = odds_2020.drop(columns = ['1st','2nd','3rd','4th','5th','6th','7th',
                                          '8th','9th','OpenOU','Unnamed: 18','Unnamed: 20', 'Close','CloseOU', 'Unnamed: 22', 'RunLine' ])
odds_2020['year'] = '2020'

In [213]:
odds_2020.dtypes

Date            int64
Rot             int64
VH             object
Team           object
Pitcher        object
Final           int64
Open            int64
away_team      object
away_open     float64
away_score    float64
year           object
dtype: object

In [214]:
odds_2020

,Date,Rot,VH,Team,Pitcher,Final,Open,away_team,away_open,away_score,year
1,401,962,H,CUB,KHENDRICK,3,-205,PIT,183.0,5.0,2020
3,401,964,H,PHI,ANOLA,3,-105,ATL,-105.0,2.0,2020
5,401,966,H,SDG,YDARVISH,8,-210,ARI,188.0,7.0,2020
7,401,968,H,COL,GMARQUEZ,8,188,LAD,-210.0,5.0,2020
9,401,970,H,CIN,LCASTILLO,6,-115,STL,105.0,11.0,2020
...,...,...,...,...,...,...,...,...,...,...,...
4915,1027,954,H,HOU,JURQUIDY,7,-115,ATL,-105.0,2.0,2020
4917,1029,956,H,ATL,IANDERSON,2,-115,HOU,-105.0,0.0,2020
4919,1030,958,H,ATL,DLEE-L,3,-115,HOU,-105.0,2.0,2020
4921,1031,960,H,ATL,TDAVIDSON-L,5,-105,HOU,-115.0,9.0,2020


In [215]:
odds_2021 = pd.read_excel('data/mlb odds 2021.xlsx')

In [216]:
odds_2021 = pd.read_excel(filepath)
odds_2021['away_team'] = odds_2021.Team.shift(1)
odds_2021['away_open'] = odds_2021['Open'].shift(1)
odds_2021['away_score'] = odds_2021['Final'].shift(1)
odds_2021 = odds_2021[odds_2021['VH'] == 'H']
odds_2021 = odds_2021.drop(columns = ['1st','2nd','3rd','4th','5th','6th','7th',
                                          '8th','9th','OpenOU','Unnamed: 18','Unnamed: 20', 'Close','CloseOU', 'Unnamed: 22', 'RunLine' ])
odds_2021['year'] = '2021'

In [217]:
odds = odds.append(odds_2020)

In [218]:
odds = odds.append(odds_2021)

In [219]:
odds = odds.drop(columns = ['Run Line', 'Unnamed: 22'])

In [220]:
odds.shape

(24642, 11)

In [221]:
odds['Date'] = odds.Date.astype(str)

In [222]:
odds['date'] = odds['Date'] + odds['year']

In [223]:
odds['date'] = odds['date'].astype(str)

In [224]:
odds['date'] = pd.to_datetime(odds['date'], format='%m%d%Y')

In [237]:
odds = odds.rename(columns={'Team':'home_abbr', 'away_team':'away_abbr', 'Open':'home_ml', 'Final':'home_score'})

In [238]:
# Looking at How odds perform:

In [239]:
odds.sort_values('home_abbr').home_abbr.unique()

array(['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL',
       'DET', 'HOU', 'KCR', 'LAA', 'LAD', 'MIA', 'MIL', 'MIN', 'NYM',
       'NYY', 'OAK', 'PHI', 'PIT', 'SDP', 'SEA', 'SFG', 'STL', 'TBR',
       'TEX', 'TOR', 'WSN'], dtype=object)

In [240]:
abbrs_no_match = {'CUB':'CHC', 'CWS':'CHW', 'KAN':'KCR', 'LOS':'LAD', 'SDG':'SDP', 'SFO':'SFG','WAS':'WSN', 'TAM':'TBR'}

In [241]:
odds = odds.replace({'home_abbr':abbrs_no_match})

In [242]:
odds=odds.replace({'away_abbr':abbrs_no_match})

In [243]:
odds.sort_values('home_abbr').home_abbr.unique()

array(['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL',
       'DET', 'HOU', 'KCR', 'LAA', 'LAD', 'MIA', 'MIL', 'MIN', 'NYM',
       'NYY', 'OAK', 'PHI', 'PIT', 'SDP', 'SEA', 'SFG', 'STL', 'TBR',
       'TEX', 'TOR', 'WSN'], dtype=object)

In [270]:
replace_nl = {'NL':np.nan}

In [272]:
odds = odds.replace({'home_ml':replace_nl})
odds = odds.replace({'home_score':replace_nl})
odds = odds.replace({'home_score':replace_nl})

In [275]:
odds['home_ml'] = odds['home_ml'].astype(int, errors='ignore')
odds['home_score'] = odds['home_score'].astype(int, errors = 'ignore')
odds['away_score'] = odds['away_score'].astype(int, errors = 'ignore')

In [282]:
odds = odds.reset_index(drop = True)

In [290]:
outcome = []
prediction = []
prob = []

for i in odds.index:
    ml = odds['home_ml'][i]
    home_score = odds['home_score'][i]
    away_score = odds['away_score'][i]
    try:
        if ml == 100:
            pass
        if ml < 0:
            prob.append(-ml/(-ml + 100))
        elif ml > 100:
            prob.append(100/(ml + 100))


        outcome.append(home_score>away_score)
        prediction.append( ml < 0 )
    except TypeError:
        pass

In [292]:
from sklearn.metrics import accuracy_score

In [295]:
accuracy_score(outcome,prediction)

0.5726228643318048

In [296]:
odds.to_csv('data/odds.csv', index=False)